# Datas Science 6 : Sécurité - RGPD
## 6. Démonstration pratique
### 6.1 Identification des données personnelles

Considérons un dataset type :

In [1]:
import pandas as pd

data = {
    'nom': ['Dupont', 'Martin', 'Bernard'],
    'email': ['j.dupont@mail.fr', 'p.martin@mail.fr', 's.bernard@mail.fr'],
    'age': [34, 28, 45],
    'code_postal': ['75015', '69001', '33000'],
    'salaire': [45000, 38000, 52000],
    'maladie_chronique': ['diabète', 'aucune', 'hypertension']
}
df = pd.DataFrame(data)

**Classification** :

- Identifiants directs ?
- Quasi-identifiants ?
- Données sensibles ?
- Données non sensibles ?

#### Réponses

- Identifiants directs : `nom`, `email`
- Quasi-identifiants : `age`, `code_postal` (combinaison potentiellement identifiante)
- Données sensibles : `maladie_chronique` (donnée de santé)
- Données non sensibles : `salaire`

### 6.2 Pseudonymisation avec hachage

#### Hachage simple

In [3]:
import hashlib

def hash_with_salt(value, salt="secret_salt_2024"):
    """Pseudonymise une valeur avec un salt."""
    salted = f"{salt}{value}".encode()
    return hashlib.sha256(salted).hexdigest()[:12]

# Application
df_pseudo = df.copy()
df_pseudo['nom'] = df_pseudo['nom'].apply(hash_with_salt)
df_pseudo['email'] = df_pseudo['email'].apply(hash_with_salt)

print(df_pseudo)

            nom         email  age code_postal  salaire maladie_chronique
0  6b5223ebdb37  099c3faedd2f   34       75015    45000           diabète
1  117d0fcbfb91  33f158747d1a   28       69001    38000            aucune
2  c6e4e8257992  4f8323438f58   45       33000    52000      hypertension


#### Hachage avec salt

In [6]:
import hashlib
import secrets

def hash_avec_salt(valeur, salt=None):
    """Hache une valeur avec un salt unique."""
    if salt is None:
        salt = secrets.token_hex(16)  # Génère un salt aléatoire
    
    salted = f"{salt}{valeur}".encode()
    hash_result = hashlib.sha256(salted).hexdigest()
    
    # On doit stocker le salt avec le hash pour pouvoir vérifier plus tard
    return {"salt": salt, "hash": hash_result}

# Exemple
email = "jean.dupont@email.fr"
resultat = hash_avec_salt(email)

In [5]:
resultat

{'salt': '4df64e23cd4d579509d98164c67fd454',
 'hash': 'd9ab01e5a3822fd5b6146e1ede71da0a717274446437834b47c2b96ac1bf1e9d'}

On ne détaille pas le code ici, mais voici une comparaison de ce qu’on obtient avec un hachage simple et un hachage « salé » :

```
SANS SALT (vulnérable)
─────────────────────────────────────────────────────────
jean.dupont@email.fr  →  SHA256  →  8f4e2a1b...
jean.dupont@email.fr  →  SHA256  →  8f4e2a1b...  (identique!)

AVEC SALT (sécurisé)
─────────────────────────────────────────────────────────
salt1 + jean.dupont@email.fr  →  SHA256  →  9f8e7d6c...
salt2 + jean.dupont@email.fr  →  SHA256  →  2a3b4c5d...  (différent!)
```


| Type | Description | Sécurité |
|------|-------------|----------|
| **Salt global** | Un seul salt secret pour toute la base | Moyen — si le salt fuite, toute la base est vulnérable |
| **Salt individuel** | Un salt unique par enregistrement | Fort — compromission limitée à un enregistrement |


### 6.3 Pseudonymisation avec token

In [ ]:
import secrets
import json

class TokenVault:
    """Coffre-fort de tokenisation (simplifié pour l'exemple)."""
    
    def __init__(self):
        self._vault = {}  # En production : base chiffrée, HSM, etc.
    
    def tokenize(self, valeur_sensible):
        """Remplace une valeur sensible par un token."""
        token = f"TKN_{secrets.token_hex(8)}"
        self._vault[token] = valeur_sensible
        return token
    
    def detokenize(self, token):
        """Retrouve la valeur originale (accès restreint !)."""
        return self._vault.get(token)

# Utilisation
vault = TokenVault()

# Tokenisation d'un numéro de carte
numero_carte = "4532-1234-5678-9012"
token = vault.tokenize(numero_carte)
print(f"Token stocké en base : {token}")  # TKN_a1b2c3d4e5f6g7h8

# Plus tard, pour débiter la carte (service autorisé uniquement)
carte_originale = vault.detokenize(token)
print(f"Numéro récupéré : {carte_originale}")  # 4532-1234-5678-9012

### 6.4 Génération données (simulées)

In [8]:
from faker import Faker

fake = Faker('fr_FR')

def generer_donnees_synthetiques(n=100):
    """Génère un dataset réaliste mais entièrement fictif."""
    data = []
    for _ in range(n):
        data.append({
            'nom': fake.name(),
            'email': fake.email(),
            'age': fake.random_int(min=18, max=80),
            'code_postal': fake.postcode(),
            'ville': fake.city()
        })
    return pd.DataFrame(data)

df_fake = generer_donnees_synthetiques(10)
print(df_fake)

                           nom                         email  age code_postal  \
0      Camille Vallet-Bousquet      launaydaniel@example.com   19       48711   
1            Émilie de Maillet       christine15@example.org   75       83701   
2  Corinne Laine de la Ferrand  stephanieleclerc@example.net   21       50386   
3                Célina Allain    charlestristan@example.com   22       30475   
4               Agnès Da Silva     anaislegendre@example.org   53       34725   
5         Luce Perrot du Marty       pagescelina@example.org   61       49470   
6               Victor Carlier          jguillon@example.org   21       87235   
7             Caroline Lefèvre    nathaliepierre@example.net   47       97475   
8              Jacques Bernier      chretienluce@example.net   36       94223   
9               Olivier Menard       josephine63@example.net   38       46479   

                 ville  
0              Guillon  
1             Lebreton  
2                Roche  
3       

### 6.4 Application du K-anonymat

In [11]:
def apply_k_anonymity(df, quasi_identifiers, k=2):
    """
    Applique le k-anonymat par généralisation.
    Simplifié pour la démonstration.
    """
    df_anon = df.copy()
    
    # Généralisation de l'âge en tranches
    if 'age' in quasi_identifiers:
        df_anon['age'] = pd.cut(df_anon['age'], 
                                bins=[0, 30, 40, 50, 60, 100],
                                labels=['<30', '30-40', '40-50', '50-60', '>60'])
    
    # Généralisation du code postal (premiers chiffres seulement)
    if 'code_postal' in quasi_identifiers:
        df_anon['code_postal'] = df_anon['code_postal'].str[:2] + '***'
    
    # Vérification du k-anonymat
    groups = df_anon.groupby(quasi_identifiers, observed=False).size()
    violations = groups[groups < k]
    
    if len(violations) > 0:
        print(f"Attention : {len(violations)} groupes < k={k}")
        print("Groupes à risque :", violations.index.tolist())
    else:
        print(f"K-anonymat respecté (k={k})")
    
    return df_anon

# Application
quasi_ids = ['age', 'code_postal']
df_kanon = apply_k_anonymity(df, quasi_ids, k=2)
print(df_kanon)

Attention : 15 groupes < k=2
Groupes à risque : [('<30', '33***'), ('<30', '69***'), ('<30', '75***'), ('30-40', '33***'), ('30-40', '69***'), ('30-40', '75***'), ('40-50', '33***'), ('40-50', '69***'), ('40-50', '75***'), ('50-60', '33***'), ('50-60', '69***'), ('50-60', '75***'), ('>60', '33***'), ('>60', '69***'), ('>60', '75***')]
       nom              email    age code_postal  salaire maladie_chronique
0   Dupont   j.dupont@mail.fr  30-40       75***    45000           diabète
1   Martin   p.martin@mail.fr    <30       69***    38000            aucune
2  Bernard  s.bernard@mail.fr  40-50       33***    52000      hypertension


### 6.5 Ré-identification

In [13]:
# Dataset "anonymisé" (sans les noms)
df_public = df[['age', 'code_postal', 'salaire']].copy()

# Attaquant avec une source externe
source_externe = pd.DataFrame({
    'nom': ['Jean Dupont'],
    'age': [34],
    'code_postal': ['75015']
})

# Attaque par jointure (retrouve les infos en croisant seulement âge et code postal)
resultat = pd.merge(source_externe, df_public, on=['age', 'code_postal'])
print("Ré-identification réussie :")
print(resultat)
# L’attaquant sait à qui correspond cet âge et ce code postal
# et donc retrouve le salaire de Jean Dupont !

Ré-identification réussie :
           nom  age code_postal  salaire
0  Jean Dupont   34       75015    45000
